# Solar Energy


Salient's historical and downscale data include the components necessary to calculate solar energy output. This notebook will show you how to obtain the source meteorological data and then convert it to energy based on array characteristics.


In [ ]:
import sys
import os

try:
    import salientsdk as sk
except ModuleNotFoundError as e:
    if os.path.exists("../salientsdk"):
        sys.path.append(os.path.abspath(".."))
        import salientsdk as sk
    else:
        raise ModuleNotFoundError("Install salient SDK with: pip install salientsdk")


# Analyze one year of data:
(start_date, end_date) = ("2022-01-01", "2022-12-31")
# When we want to plot a timeseries, focus on a specific month:
plt_time = slice("2022-07-02", "2022-07-05")

sk.set_file_destination("solar_example")
sk.login("username", "password")

## Get Solar Meteorological Data


### Set geographic bounds

The Salient SDK uses a `Location` object to specify the geographic bounds of a request. Let's analyze solar energy at 3 sites in Maine, Texas, and California. We can also predict solar energy over a gridded area defined by a polygon shapefile.


In [ ]:
if True:  # Analyze solar at 3 locations with a location_file
    loc_file = sk.upload_location_file(
        lats=[44.3327, 31.2194, 34.830556],
        lons=[-69.781, -102.1922, -118.398056],
        names=["3 Corners ME", "Roadrunner TX", "Solar Star CA"],
        rated_capacity=[109, 400, 579],  # MW
        geoname="solar_example",
        force=False,
    )
    loc = sk.Location(location_file=loc_file)

else:  # Analyze solar over the Western ERCOT region with a shapefile
    coords = [
        (-103.25, 32.00),
        (-105.50, 32.00),
        (-105.50, 31.00),
        (-104.50, 29.50),
        (-103.25, 29.00),
        (-102.50, 29.75),
        (-101.25, 29.50),
        (-100.50, 29.00),
        (-98.000, 34.00),
        (-100.00, 34.50),
        (-100.00, 35.75),
        (-103.25, 35.75),
    ]

    shape_file = sk.upload_shapefile(coords, "ercot_west", force=False)
    loc = sk.Location(shapefile=shape_file)

### Get Historical Observed Data


The `data_timeseries_solar` function is a convenience function that calls `data_timeseries` to generate an hourly historical timeseries with the appropriate weather inputs for solar analysis.


In [ ]:
hst = sk.solar.data_timeseries_solar(loc=loc, start=start_date, end=end_date)
print(hst)
hst["tsi"].sel(time=plt_time).plot.line(x="time");

<xarray.Dataset> Size: 1MB
Dimensions:    (time: 8737, location: 3)
Coordinates:
  * time       (time) datetime64[ns] 70kB 2022-01-01 ... 2022-12-31
  * location   (location) object 24B '3 Corners ME' ... 'Solar Star CA'
    lat        (location) float64 24B 44.33 31.22 34.83
    lon        (location) float64 24B -69.78 -102.2 -118.4
Data variables:
    temp       (time, location) float64 210kB 1.2 12.66 6.364 ... 11.02 11.96
    wspd       (time, location) float64 210kB 2.016 5.742 4.568 ... 1.688 3.549
    tsi        (time, location) float64 210kB 0.0 17.94 160.3 ... 30.51 114.4
    dhi        (time, location) float64 210kB 0.0 17.63 60.63 ... 14.85 68.77
    dni        (time, location) float64 210kB 0.0 4.494 443.3 ... 232.1 204.7
    elevation  (location) float32 12B 92.69 780.5 975.8


## Sun to Power


`run_pvlib_dataset` uses NREL's `pvlib` to convert the meteorological dataset into a timeseries with `ac` and `dc` watts.


In [ ]:
pwr = sk.solar.run_pvlib_dataset(hst)
print(pwr)
pwr["ac"].sel(time=plt_time).plot.line(x="time");

<xarray.Dataset> Size: 699kB
Dimensions:               (time: 8737, location: 3)
Coordinates:
  * time                  (time) datetime64[ns] 70kB 2022-01-01 ... 2022-12-31
  * location              (location) object 24B '3 Corners ME' ... 'Solar Sta...
    lat                   (location) float64 24B 44.33 31.22 34.83
    lon                   (location) float64 24B -69.78 -102.2 -118.4
Data variables:
    ac                    (location, time) float64 210kB 0.0 0.0 ... 0.1569
    dc                    (location, time) float64 210kB 0.0 0.0 ... 0.1779
    effective_irradiance  (location, time) float64 210kB 0.0 0.0 ... 131.0 172.0
